In [1]:
from wdro_mp_sp_final import *
from configs_chem_process import *
import time
from tqdm import tqdm
import gurobipy as gp
from gurobipy import GRB

In [2]:
#### given the input data, and the matrix list, to generate a list of data samples in the uncertainty set####
def data_filter(in_data, D_list,d_list):
    n = len(in_data)
    K = len(D_list)
    new_data = []
    status_list = []
    for i in range(n):
        sample = in_data[i]
        sample_status = False
        diff_list = []
        for j in range(K):
            diff = D_list[j]@sample - d_list[j]
            diff_list.append(np.max(diff))

        if min(diff_list) <= 0:
            sample_status = True
            new_data.append(sample)
    return np.array(new_data)

In [3]:
lb_data_list = []
ub_data_list = []
mean_data_list = []
var_data_list = []
for t in range(set_num):
    mu_demand = mu_demand_list[t]/u_scale
    var_demand = var_demand_list[t]/u_scale
    mu_supply = mu_supply_list[t]/u_scale
    var_supply = var_supply_list[t]/u_scale
    
    mean = np.tile(np.concatenate((mu_supply, mu_demand)), period_num)
    var = np.tile(np.concatenate((var_supply, var_demand)), period_num)
    
    lb = np.tile(np.concatenate((mu_supply - var_supply, mu_demand - var_demand)), period_num)
    ub = np.tile(np.concatenate((mu_supply + var_supply, mu_demand + var_demand)), period_num)
    lb_data_list.append(lb)
    ub_data_list.append(ub) 

In [4]:
lb_data_list

[array([110.,  88.,  25.,  41., 110.,  88.,  25.,  41., 110.,  88.,  25.,
         41., 110.,  88.,  25.,  41., 110.,  88.,  25.,  41.]),
 array([64., 56., 30., 27., 64., 56., 30., 27., 64., 56., 30., 27., 64.,
        56., 30., 27., 64., 56., 30., 27.]),
 array([55., 25., 17.,  6., 55., 25., 17.,  6., 55., 25., 17.,  6., 55.,
        25., 17.,  6., 55., 25., 17.,  6.]),
 array([48., 48., 23., 23., 48., 48., 23., 23., 48., 48., 23., 23., 48.,
        48., 23., 23., 48., 48., 23., 23.])]

In [5]:
ub_data_list

[array([130., 112.,  35.,  49., 130., 112.,  35.,  49., 130., 112.,  35.,
         49., 130., 112.,  35.,  49., 130., 112.,  35.,  49.]),
 array([76., 68., 40., 37., 76., 68., 40., 37., 76., 68., 40., 37., 76.,
        68., 40., 37., 76., 68., 40., 37.]),
 array([65., 35., 23., 14., 65., 35., 23., 14., 65., 35., 23., 14., 65.,
        35., 23., 14., 65., 35., 23., 14.]),
 array([52., 52., 27., 27., 52., 52., 27., 27., 52., 52., 27., 27., 52.,
        52., 27., 27., 52., 52., 27., 27.])]

In [6]:
np.random.seed(5555)
data_size = 1000
data_list = np.zeros((1, len(ind_u)))
for t in range(len(pbar)):
    data = np.random.uniform(lb_data_list[t], ub_data_list[t], (int(data_size*pbar[t]), len(ind_u)))
    data_list = np.concatenate((data_list, data), axis = 0)

U_sample = data_filter(data_list, D_set, d_set)

In [7]:
### Algorithm of WDRO_final ###



import time
ccg_str = time.time()

UB_list = np.array([])
LB_list = np.array([])

#add_u_opt = "worst"
add_u_opt = "worst"

epsilon = 0.02
num_iter = 50

UB = 1e8
LB = -1e7

N = len(U_sample)

u_set = []

if add_u_opt == "worst":
    u_set = [U_sample]
elif add_u_opt == "all":
    _u_set = []
    for t in range(N):
        _u_set.append(np.array(u_set_init))
    u_set = [_u_set]


for k_iter in tqdm(range(num_iter)):

    if add_u_opt == "worst":
        lamda_val, x, z_data, mp = MP_WDRO_v3(u_set, k_iter, U_sample)  #### solve master problem
    elif add_u_opt == "all":
        lamda_val, x, z_data, mp = MP_WDRO_v4(u_set, k_iter, U_sample)
        
    print("master problem termination condition: ", mp.status)
        
    LB = mp.objVal    
    LB_list = np.append(LB_list, LB)
    
    sp_obj_vec = np.array([])

    ### for storing the uncertainty scenarios and the objective function stored for iterating all z_data
    u_set_iter = []
    sp_obj_iter = []
    
    for t in range(len(z_data)):
        
        z_iter = z_data[t]    
        sp_obj_vec = np.array([])  # objective value vector for K subproblems
        u_list = []                # worst uncertainty scenario for K subproblems
        for k in ind_k:
            D, d = D_set[k], d_set[k]
            sp_obj, sp_u, sp_vio, sp = SP_WDRO_v3(x, D, d, z_iter)  ## solve k different subproblems
                        
            u_list.append(sp_u)
            sp_obj_vec = np.append(sp_obj_vec, sp_obj)

        sp_obj_max = np.max(sp_obj_vec)
        u_max = u_list[np.argmax(sp_obj_vec)]

        if add_u_opt == "worst":
            u_set_iter.append(u_max)
        elif add_u_opt == "all":
            u_set_iter.append(np.array(u_list))
            
        sp_obj_iter.append(sp_obj_max)
        
    u_set.append(np.array(u_set_iter))

    UB_candidate = (np.dot(c,x) + lamda_val*eps + 1/N*sum(sp_obj_iter) + 1/N*sum( np.dot(z_data[i,:],U_sample[i,:]) for i in range(N)) )
    print("UB candidate is: ", UB_candidate)

    UB = min(UB, UB_candidate)
    UB_list = np.append(UB_list,UB)
    
    print("Lower bound is: ", LB)
    print("Upper bound is: ", UB)

            
    if np.abs(UB - LB)/abs(LB) <= epsilon:
        print("first stage decision variables: ", x)
        break

computation_time_ccg = time.time() - ccg_str
print("computation time of DRO ccg is: ", computation_time_ccg)


import matplotlib.pyplot as plt

x_label = range(len(LB_list))
plt.scatter(x_label, UB_list)
plt.scatter(x_label, LB_list)
print("upper bound list is: ", UB_list)
print("lower bound list is: ", LB_list)

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

Set parameter Username
Academic license - for non-commercial use only - expires 2026-01-22
master problem termination condition:  2


  2%|█▌                                                                            | 1/50 [30:43<25:05:41, 1843.69s/it]

UB candidate is:  -218.1611896773691
Lower bound is:  -461.5618114049944
Upper bound is:  -218.1611896773691
master problem termination condition:  2


  4%|███                                                                         | 2/50 [1:01:19<24:31:16, 1839.09s/it]

...

UB candidate is:  1916.4602294793135
Lower bound is:  -462.3920565167095
Upper bound is:  -218.1611896773691
master problem termination condition:  2


 76%|████████████████████████████████████████████████████████▏                 | 38/50 [29:41:55<17:20:06, 5200.57s/it]

UB candidate is:  3077.866019090037
Lower bound is:  -462.3920565167095
Upper bound is:  -218.1611896773691


 76%|█████████████████████████████████████████████████████████                  | 38/50 [29:55:12<9:26:54, 2834.53s/it]


GurobiError: Out of memory

In [ ]:
c.reshape(10,-1)